In [55]:
import json
import numpy as np
import skimage
import tifffile
import os
import shutil

In [89]:
original_image_dir = '/disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images'  # Where your original images are stored
json_file = '/disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/_annotations.coco.json'

mask_output_folder = '/disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels'  # Modify this as needed. Using val2 so my data is not overwritten

In [90]:
with open(json_file, 'r') as f:
    data = json.load(f)

#--------------------------------
images = data['images']
annotations = data['annotations']
categories = data['categories']
#--------------------------------
display(categories)

[{'id': 0, 'name': 'M2A1Slammer', 'supercategory': 'none'},
 {'id': 1, 'name': 'M2A1Slammer', 'supercategory': 'M2A1Slammer'},
 {'id': 2, 'name': 'M5SandstormMLRS', 'supercategory': 'M2A1Slammer'},
 {'id': 3, 'name': 'T140Angara', 'supercategory': 'M2A1Slammer'},
 {'id': 4, 'name': 'ZamakMRL', 'supercategory': 'M2A1Slammer'}]

In [91]:
categories[0]['name'] = "background"
categories[0]['supercategory'] = "background"

In [92]:
display(categories)

[{'id': 0, 'name': 'background', 'supercategory': 'background'},
 {'id': 1, 'name': 'M2A1Slammer', 'supercategory': 'M2A1Slammer'},
 {'id': 2, 'name': 'M5SandstormMLRS', 'supercategory': 'M2A1Slammer'},
 {'id': 3, 'name': 'T140Angara', 'supercategory': 'M2A1Slammer'},
 {'id': 4, 'name': 'ZamakMRL', 'supercategory': 'M2A1Slammer'}]

In [93]:
def create_mask(image_info, annotations, output_folder):
    # Create an empty mask as a numpy array
    mask_np = np.zeros((image_info['height'], image_info['width']), dtype=np.uint16)

    for ann in annotations:
        if ann['image_id'] == image_info['id']:
                print("#----- : ",ann['image_id'])
                # Extract segmentation polygon
                #ann['category_id'] +=1
                category_id = ann['category_id'] 
                
                for idx,seg in enumerate(ann['segmentation']):
                    # Convert polygons to a binary mask and add it to the main mask
                    rr, cc = skimage.draw.polygon(seg[1::2], seg[0::2], mask_np.shape)
                    mask_np[rr, cc] = category_id
                    
                    #object_number += 1 #We are assigning each object a unique integer value (labeled mask)

    # Save the numpy array as a TIFF using tifffile library
    print(image_info['file_name'].split(".")[0].replace("_png",".png"))
    mask_path = os.path.join(output_folder, image_info['file_name'].split(".")[0].replace("_png",".png"))
    tifffile.imsave(mask_path, mask_np)

    image_info['file_name'] = image_info['file_name'].split(".")[0].replace("_png",".png") 

    #----

    print(f"Saved mask for {image_info['file_name']} to {mask_path}")

    return annotations


In [94]:
for img in images:
    # Create the masks
    create_mask(img, annotations, mask_output_folder)



#----- :  0
T140Angara__part_19.png
Saved mask for T140Angara__part_19.png to /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_19.png
#----- :  1
#----- :  1
#----- :  1
#----- :  1
M2A1Slammer2__part_17.png
Saved mask for M2A1Slammer2__part_17.png to /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M2A1Slammer2__part_17.png
#----- :  2
T140Angara__part_25.png
Saved mask for T140Angara__part_25.png to /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_25.png
#----- :  3
Zamak__part_21.png
Saved mask for Zamak__part_21.png to /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/Zamak__part_21.png
#----- :  4
Zamak__part_23.png
Saved mask for Zamak__part_23.png to /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/Zamak__part_23.png
#----- :  5
#----- :  5
M2A1Slammer2__part_11.png
Saved mask for M2A1Slammer2__part_11.png to /disk3/eric/dataset/VISI

/tmp/ipykernel_612021/4107398179.py:22: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(mask_path, mask_np)


In [95]:
len(os.listdir(original_image_dir))

81

In [96]:
len(mask_output_folder)

len(os.listdir(mask_output_folder))

81

In [97]:
#-- save json 
# 원본파일에 덮어씀

with open(json_file, 'w') as file:
    json.dump(data, file)
